The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install -U mxnet-cu101mkl==1.6.0  # updating mxnet to at least v1.6


# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our objective function. 
After training, we will need these parameters 
in order to make future predictions.
Additionally, we will sometimes wish 
to extract the parameters 
either to reuse them in some other context,
to save our model to disk so that 
it may be executed in other software,
or for examination in the hopes of 
gaining scientific understanding.

Most of the time, we will be able 
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on the framework
to do the heavy lifting.
However, when we move away from 
stacked architectures with standard layers, 
we will sometimes need to get into the weeds
of declaring and manipulating parameters. 
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Parameter initialization.
* Sharing parameters across different model components.

We start by focusing on an MLP with one hidden layer.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-1-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-1-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-1-0">
```

In [1]:
from mxnet import init, np, npx
from mxnet.gluon import nn
npx.set_np()

net = nn.Sequential()
net.add(nn.Dense(8, activation='relu'))
net.add(nn.Dense(1))
net.initialize()  # Use the default initialization method

x = np.random.uniform(size=(2, 4))
net(x)  # Forward computation

array([[0.0054572 ],
       [0.00488594]])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-1-1">
```

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
x = torch.randn(2, 4)
net(x)

tensor([[-0.4548],
        [-0.1430]], grad_fn=<AddmmBackward>)

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

## Parameter Access

Let us start with how to access parameters
from the models that you already know.
When a model is defined via the Sequential class,
we can first access any layer by indexing 
into the model as though it were a list.
Each layer's parameters are conveniently 
located in its attribute. 
We can inspect the parameters of the `net` defined above as a dictionary.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-3-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-3-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-3-0">
```

In [2]:
print(net[0].params)
print(net[1].params)

dense0_ (
  Parameter dense0_weight (shape=(8, 4), dtype=float32)
  Parameter dense0_bias (shape=(8,), dtype=float32)
)
dense1_ (
  Parameter dense1_weight (shape=(1, 8), dtype=float32)
  Parameter dense1_bias (shape=(1,), dtype=float32)
)


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-3-1">
```

In [2]:
print(net[2].state_dict())  

OrderedDict([('weight', tensor([[-0.3043, -0.3100, -0.1816, -0.2061, -0.2144, -0.3175, -0.0304,  0.1868]])), ('bias', tensor([-0.0556]))])


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

The output tells us a few important things.
First, each fully-connected layer 
contains two parameters, e.g., 
`weight` and `bias` (may with prefix),
corresponding to that layer's 
weights and biases, respectively.
Both are stored as single precision floats.
Note that the names of the parameters
allow us to *uniquely* identify
each layer's parameters,
even in a network containing hundreds of layers.


### Targeted Parameters

Note that each parameter is represented
as an instance of the `Parameter` class.
To do anything useful with the parameters,
we first need to access the underlying numerical values. 
There are several ways to do this.
Some are simpler while others are more general.
To begin, given a layer, 
we can access one of its parameters 
via the `bias` or `weight` attributes, which returns an `Parameter` instance
and further access that parameter's value
via its `data` method.
The following code extracts the bias
from the second neural network layer.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-5-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-5-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-5-0">
```

In [3]:
print(type(net[1].bias))
print(net[1].bias)
print(net[1].bias.data())

<class 'mxnet.gluon.parameter.Parameter'>
Parameter dense1_bias (shape=(1,), dtype=float32)
[0.]


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-5-1">
```

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0556], requires_grad=True)
tensor([-0.0556])


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

Parameters are complex objects,
containing data, gradients,
and additional information.
That's why we need to request the data explicitly.


In addition to `data`, each `Parameter` also provides a `grad` method for accessing the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-7-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-7-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-7-0">
```

In [4]:
net[0].weight.grad()

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-7-1">
```

In [4]:
net[0].weight.grad == None

True

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

### All Parameters at Once

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex blocks, (e.g., nested Blocks),
since we would need to recurse 


through the entire tree in to extract
each sub-block's parameters.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-9-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-9-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-9-0">
```

In [5]:
# parameters only for the first layer
print(net[0].collect_params())
# parameters of the entire network
print(net.collect_params())

dense0_ (
  Parameter dense0_weight (shape=(8, 4), dtype=float32)
  Parameter dense0_bias (shape=(8,), dtype=float32)
)
sequential0_ (
  Parameter dense0_weight (shape=(8, 4), dtype=float32)
  Parameter dense0_bias (shape=(8,), dtype=float32)
  Parameter dense1_weight (shape=(1, 8), dtype=float32)
  Parameter dense1_bias (shape=(1,), dtype=float32)
)


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-9-1">
```

In [5]:
# parameters only for the first layer
print(net[0].state_dict())
# parameters of the entire network
print(net.state_dict())

OrderedDict([('weight', tensor([[ 0.2579, -0.3110, -0.1081,  0.1590],
        [-0.0491,  0.4084, -0.3703,  0.3920],
        [-0.2374,  0.4125,  0.1550,  0.0914],
        [ 0.4156, -0.0587, -0.0885,  0.1579],
        [ 0.1179, -0.4336,  0.3640, -0.4356],
        [ 0.0524,  0.0702,  0.4883,  0.1292],
        [ 0.1694,  0.3118,  0.1506, -0.4303],
        [ 0.0736, -0.1570,  0.4526, -0.2570]])), ('bias', tensor([ 0.2401,  0.2881,  0.0832,  0.0763, -0.2688,  0.3791,  0.2861, -0.4448]))])
OrderedDict([('0.weight', tensor([[ 0.2579, -0.3110, -0.1081,  0.1590],
        [-0.0491,  0.4084, -0.3703,  0.3920],
        [-0.2374,  0.4125,  0.1550,  0.0914],
        [ 0.4156, -0.0587, -0.0885,  0.1579],
        [ 0.1179, -0.4336,  0.3640, -0.4356],
        [ 0.0524,  0.0702,  0.4883,  0.1292],
        [ 0.1694,  0.3118,  0.1506, -0.4303],
        [ 0.0736, -0.1570,  0.4526, -0.2570]])), ('0.bias', tensor([ 0.2401,  0.2881,  0.0832,  0.0763, -0.2688,  0.3791,  0.2861, -0.4448])), ('2.weight', tensor([

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

This provides us with another way of accessing the parameters of the network:

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-11-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-11-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-11-0">
```

In [6]:
net.collect_params()['dense1_bias'].data()

array([0.])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-11-1">
```

In [6]:
net.state_dict()['2.bias'].data

tensor([-0.0556])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-12-0" class="mdl-tabs__tab is-active">mxnet</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-12-0">
```

Throughout the book we encounter Blocks 
that name their sub-Blocks in various ways. 
Sequential simply numbers them.
We can exploit this naming convention by leveraging
one clever feature of `collect_params`:
it allows us to filter the parameters 
returned by using regular expressions.

In [7]:
print(net.collect_params('.*weight'))
print(net.collect_params('dense0.*'))

sequential0_ (
  Parameter dense0_weight (shape=(8, 4), dtype=float32)
  Parameter dense1_weight (shape=(1, 8), dtype=float32)
)
sequential0_ (
  Parameter dense0_weight (shape=(8, 4), dtype=float32)
  Parameter dense0_bias (shape=(8,), dtype=float32)
)


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

### Collecting Parameters from Nested Blocks

Let us see how the parameter naming conventions work 
if we nest multiple blocks inside each other. 
For that we first define a function that produces blocks 
(a block factory, so to speak) and then 
combine these inside yet larger blocks.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-14-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-14-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-14-0">
```

In [8]:
def block1():
    net = nn.Sequential()
    net.add(nn.Dense(32, activation='relu'))
    net.add(nn.Dense(16, activation='relu'))
    return net

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add(block1())
    return net

rgnet = nn.Sequential()
rgnet.add(block2())
rgnet.add(nn.Dense(10))
rgnet.initialize()
rgnet(x)

array([[-6.3465841e-09, -1.1096747e-09,  6.4161774e-09,  6.6354131e-09,
        -1.1265502e-09,  1.3285141e-10,  9.3619361e-09,  3.2229091e-09,
         5.9429857e-09,  8.8181418e-09],
       [-8.6219423e-09, -7.5150813e-10,  8.3133243e-09,  8.9321119e-09,
        -1.6739999e-09,  3.2406069e-10,  1.2115976e-08,  4.4926454e-09,
         8.0741742e-09,  1.2075874e-08]])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-14-1">
```

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(x)

tensor([[-0.0419],
        [-0.0406]], grad_fn=<AddmmBackward>)

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

Now that we have designed the network, 
let us see how it is organized.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-16-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-16-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-16-0">
```

In [9]:
print(rgnet.collect_params)
print(rgnet.collect_params())

<bound method Block.collect_params of Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Dense(4 -> 32, Activation(relu))
      (1): Dense(32 -> 16, Activation(relu))
    )
    (1): Sequential(
      (0): Dense(16 -> 32, Activation(relu))
      (1): Dense(32 -> 16, Activation(relu))
    )
    (2): Sequential(
      (0): Dense(16 -> 32, Activation(relu))
      (1): Dense(32 -> 16, Activation(relu))
    )
    (3): Sequential(
      (0): Dense(16 -> 32, Activation(relu))
      (1): Dense(32 -> 16, Activation(relu))
    )
  )
  (1): Dense(16 -> 10, linear)
)>
sequential1_ (
  Parameter dense2_weight (shape=(32, 4), dtype=float32)
  Parameter dense2_bias (shape=(32,), dtype=float32)
  Parameter dense3_weight (shape=(16, 32), dtype=float32)
  Parameter dense3_bias (shape=(16,), dtype=float32)
  Parameter dense4_weight (shape=(32, 16), dtype=float32)
  Parameter dense4_bias (shape=(32,), dtype=float32)
  Parameter dense5_weight (shape=(16, 32), dtype=float32)
  Parameter dense5_bi

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-16-1">
```

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

Since the layers are hierarchically nested,
we can also access them as though 
indexing through nested lists. 
For instance, we can access the first major block, 
within it the second subblock, 
and within that the bias of the first layer,
with as follows:

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-18-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-18-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-18-0">
```

In [10]:
rgnet[0][1][0].bias.data()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-18-1">
```

In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.3797,  0.0444, -0.2120, -0.4818,  0.4489, -0.3303,  0.3874,  0.4557])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

## Parameter Initialization

Now that we know how to access the parameters,
let us look at how to initialize them properly.
We discussed the need for initialization in :numref:`sec_numerical_stability`. 


```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-20-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-20-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-20-0">
```

By default, MXNet initializes weight matrices
uniformly by drawing from $U[-0.07, 0.07]$ 
and the bias parameters are all set to $0$.
However, we will often want to initialize our weights
according to various other protocols. 
MXNet's `init` module provides a variety 
of preset initialization methods.
If we want to create a custom initializer,
we need to do some extra work.

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-20-1">
```

By default, PyTorch initializes weight and bias matrices
uniformly by drawing from a range that is computed according to the input and output dimension. 
However, we will often want to initialize our weights
according to various other protocols. 
PyTorch's `nn.init` module provides a variety 
of preset initialization methods.
If we want to create a custom initializer,
we need to do some extra work.

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```


### Built-in Initialization

Let us begin by calling on built-in initializers. 
The code below initializes all weight parameters 
as Gaussian random variables 
with standard deviation $.01$, while bias parameters set to 0.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-22-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-22-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-22-0">
```

In [11]:
# force_reinit ensures that variables are freshly initialized
# even if they were already initialized previously
net.initialize(init=init.Normal(sigma=0.01), force_reinit=True)
net[0].weight.data()[0]

array([-0.00324057, -0.00895028, -0.00698632,  0.01030831])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-22-1">
```

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(net[0].weight, mean=0, std=0.01)
        nn.init.zeros_(net[0].bias)
net.apply(init_normal)    
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0070, -0.0111, -0.0073, -0.0047]), tensor(0.))

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

We can also initialize all parameters 
to a given constant value (say, $1$).

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-24-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-24-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-24-0">
```

In [12]:
net.initialize(init=init.Constant(1), force_reinit=True)
net[0].weight.data()[0]

array([1., 1., 1., 1.])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-24-1">
```

In [11]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.constant_(net[0].weight, 1)
        nn.init.zeros_(net[0].bias)
net.apply(init_normal)    
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

We can also apply different initializers for certain Blocks.
For example, below we initialize the first layer
with the `Xavier` initializer
and initialize the second layer 
to a constant value of 42.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-26-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-26-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-26-0">
```

In [13]:
net[0].weight.initialize(init=init.Xavier(), force_reinit=True)
net[1].initialize(init=init.Constant(42), force_reinit=True)
print(net[0].weight.data()[0])
print(net[1].weight.data())

[-0.17594433  0.02314097 -0.1992535   0.09509248]
[[42. 42. 42. 42. 42. 42. 42. 42.]]


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-26-1">
```

In [12]:
def xavier(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.3065, -0.5970, -0.4734, -0.4356])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

### Custom Initialization

Sometimes, the initialization methods we need 
are not provided in the `init` module. 
In the example below, we define an initializer
for the following strange distribution:

$$
\begin{aligned}
    w \sim \begin{cases}
        U[5, 10] & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U[-10, -5] & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-28-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-28-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-28-0">
```


Here we define a subclass of `Initializer`. 
Usually, we only need to implement the `_init_weight` function
which takes an `ndarray` argument (`data`) 
and assigns to it the desired initialized values. 


In [14]:
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape)
        data[:] = np.random.uniform(-10, 10, data.shape)
        data *= np.abs(data) >= 5

net.initialize(MyInit(), force_reinit=True)
net[0].weight.data()[0:2]

Init dense0_weight (8, 4)
Init dense1_weight (1, 8)


array([[ 0.       , -0.       , -0.       ,  8.522827 ],
       [ 0.       , -8.828651 , -0.       , -5.6012006]])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-28-1">
```

Again, we implement a `my_init` function to apply to `net`.

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5    
    
net.apply(my_init)
net[0].weight[0:2]

tensor([[-6.8188, -9.0834,  5.7821, -7.7177],
        [ 6.8377,  8.7972, -0.0000,  9.7700]], grad_fn=<SliceBackward>)

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

Note that we always have the option 
of setting parameters directly by calling `data` 
to access the underlying data. 


```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-30-0" class="mdl-tabs__tab is-active">mxnet</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-30-0">
```

A note for advanced users: 
if you want to adjust parameters within an `autograd` scope,
you need to use `set_data` to avoid confusing 
the automatic differentiation mechanics.

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-31-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-31-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-31-0">
```

In [15]:
net[0].weight.data()[:] += 1
net[0].weight.data()[0, 0] = 42
net[0].weight.data()[0]

array([42.      ,  1.      ,  1.      ,  9.522827])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-31-1">
```

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -8.0834,  6.7821, -6.7177])

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

## Tied Parameters

Often, we want to share parameters across multiple layers.
Later we will see that when learning word embeddings,
it might be sensible to use the same parameters
both for encoding and decoding words. 
We discussed one such case when we introduced :numref:`sec_model_construction`. 
Let us see how to do this a bit more elegantly. 
In the following we allocate a dense layer 
and then use its parameters specifically 
to set those of another layer.

```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-33-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-33-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-33-0">
```

In [16]:
net = nn.Sequential()
# We need to give the shared layer a name such that we can reference its
# parameters
shared = nn.Dense(8, activation='relu')
net.add(nn.Dense(8, activation='relu'),
        shared,
        nn.Dense(8, activation='relu', params=shared.params),
        nn.Dense(10))
net.initialize()

x = np.random.uniform(size=(2, 20))
net(x)

# Check whether the parameters are the same
print(net[1].weight.data()[0] == net[2].weight.data()[0])
net[1].weight.data()[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[1].weight.data()[0] == net[2].weight.data()[0])

[ True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True]


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-33-1">
```

In [15]:
# We need to give the shared layer a name such that we can reference its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                    shared, nn.ReLU(), 
                    shared, nn.ReLU(), 
                    nn.Linear(8, 1))
net(x)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```

This example shows that the parameters 
of the second and third layer are tied. 
They are not just equal, they are 
represented by the same exact ndarray. 
Thus, if we change one of the parameters,
the other one changes, too. 
You might wonder, 
*when parameters are tied
what happens to the gradients?*
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.

## Summary

* We have several ways to access, initialize, and tie model parameters.
* We can use custom initialization.


## Exercises

1. Use the FancyMLP defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Look at the `init` module document to explore different initializers.
1. Construct a multilayer perceptron containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?


```eval_rst

.. raw:: html

    <div class="mdl-tabs mdl-js-tabs mdl-js-ripple-effect"><div class="mdl-tabs__tab-bar text"><a href="#mxnet-35-0" class="mdl-tabs__tab is-active">mxnet</a><a href="#pytorch-35-1" class="mdl-tabs__tab ">pytorch</a></div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel is-active" id="mxnet-35-0">
```

[Discussions](https://discuss.d2l.ai/t/56)

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    <div class="mdl-tabs__panel " id="pytorch-35-1">
```

[Discussions](https://discuss.d2l.ai/t/57)

```eval_rst
.. raw:: html

    </div>
```

```eval_rst
.. raw:: html

    </div>
```